In [1]:
import pandas as pd
from MC_LIME import *


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
NUM_WEEKS=5

# True if we want to get MC_LIME explanations
MC_LIME_bool=True
course="geomatique_003"

In [4]:
# CHANGE HERE THE FEATURE SET
feature_set=f"features/{course}_features_{str(NUM_WEEKS)}_weeks.csv"
features = pd.read_csv(feature_set)
features.shape

(453, 180)

In [5]:
# GET LIME MODEL
model_path = 'models/'
model_name = model_path + "lstm_bi_" + course + "_" + str(NUM_WEEKS) + "_weeks"

loaded_model = keras.models.load_model(model_name)
config = loaded_model.get_config() # Config information about the model
print(config["layers"][0]["config"]["batch_input_shape"]) # model shape

2024-07-01 18:43:21.532279: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-07-01 18:43:21.532303: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-07-01 18:43:21.532308: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-07-01 18:43:21.532526: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-07-01 18:43:21.532547: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-07-01 18:43:22.795218: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outpu

(None, 180)


In [15]:
############################################## CODE TO GET THE INDICES OF THE FEATURES
# I pick any feature set here and take the indices that are not easy fail
index=pd.read_csv("../easy-fail/eq_week-chen_cui-" + str(course) + "/feature_labels.csv",)
index

# Make the index a column called 'feature_index'
index.reset_index(inplace=True)
index.rename(columns={'index': 'feature_index'}, inplace=True)

# Rename 'Unnamed: 0' to 'original_index'
index.rename(columns={'Unnamed: 0': 'original_index'}, inplace=True)

# Remove the other columns except 'feature_index' and 'original_index'
index = index[['feature_index', 'original_index']]

samples_id=pd.read_csv("data/"+str(course) + "_sample_5.csv")

# USE THIS INDEX MAPPING TO GET THE INDEX IN THE FEATURES
index_mapping = dict(zip(index['original_index'], index['feature_index']))
index_mapping

samples_id['feature_index'] = samples_id['original_index'].map(index_mapping)

instances=samples_id['feature_index'].values
assert np.all(instances < features.shape[0])

In [26]:
# import json
# # Specify the file path
# file_path = 'data/MC_LIME_results.json'

# # Read the JSON file
# with open(file_path, 'r') as f:
#     data = json.load(f)

# # Print the data
# list(data.keys())

In [16]:
instances

array([291, 243, 138, 174, 234])

226

### Calculate MC-Lime for each student

In [13]:
lime_path=f"uniform_eq_results_ori_{str(NUM_WEEKS)}_weeks/LIME/{str(course)}/dataframes/"
mapping={"pass" : 0, "fail" : 1}

In [18]:
# ONLY FOR ONE INDEX
import json



MC_LIME_results={}
index=index_mapping[802]
print(index)
important_features, student_features, step_sizes, y_orig = get_lime_results(features, lime_path, index, mapping)
average_values = features[important_features.index.values].mean()

MC_LIME_results[index] = MC_LIME(important_features, student_features, step_sizes, average_values, loaded_model, y_orig, debug=True)

MC_LIME_results[index] = {str(inner_key): list(inner_value) for inner_key, inner_value in MC_LIME_results[index].items()}

# Save MC_LIME_results as a JSON file
with open('MC_LIME_temp.json', 'w') as f:
    json.dump(MC_LIME_results, f)

226



___NEW GROUP SIZE___
1
{}


/Users/davide/Desktop/EPFL/XAI Semester Project/llm-xai/MC_LIME.py:81: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_orig=mapping[results["real value"][0]]
2024-07-01 18:51:42.439012: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-07-01 18:51:42.870150: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-07-01 18:51:42.928639: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-07-01 18:51:43.058849: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-07-01 18:51:43.117884: I tens

In [33]:
# MULTIPLE INDICES
import json

MC_LIME_results={}
for index in instances:
    print(index)
    important_features, student_features, step_sizes, y_orig = get_lime_results(features, lime_path, index, mapping)
    average_values = features[important_features.index.values].mean()

    MC_LIME_results[str(index)] = MC_LIME(important_features, student_features, step_sizes, average_values, loaded_model, y_orig, debug=True, max_group_size=5)
 
    MC_LIME_results[str(index)] = {str(inner_key): list(inner_value) for inner_key, inner_value in MC_LIME_results[str(index)].items()}

    # Save MC_LIME_results as a JSON file
    with open(f'MC_LIME_temp_{str(course)}.json', 'w') as f:
        json.dump(MC_LIME_results, f)

TypeError: 'numpy.int64' object is not iterable

In [9]:
important_features, student_features, step_sizes, y_orig=get_lime_results(features, lime_path, 123, mapping)

Number of features with zero step size: 0


/Users/davide/Desktop/EPFL/XAI Semester Project/llm-xai/MC_LIME.py:89: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_orig=mapping[results["real value"][0]]


In [51]:
MC_LIME_results

{123: {('competency_alignment_InWeek5',): (0.08833548027973,),
  ('competency_alignment_InWeek4',): (0.12036466410653988,),
  ('total_clicks_problem_InWeek5',): (0.09333636456076931,),
  ('competency_strength_InWeek4',): (0.3967560075134585,),
  ('time_between_sessions_std_InWeek5',): (0.06969065040014832,),
  ('total_clicks_problem_InWeek4',): (0.002398876613832389,),
  ('student_shape_InWeek3',): (0.025032824042274963,),
  ('competency_alignment_InWeek3',): (0.09505611287746726,),
  ('competency_anticipation_InWeek2',): (0.19902091659690474,),
  ('competency_alignment_InWeek2',): (0.10349531214844872,),
  ('competency_anticipation_InWeek5',): (0.24053339874792015,),
  ('time_in__problem_sum_InWeek5',): (0.28784845133620274,),
  ('regularity_periodicity_m1_InWeek5',): (0.1718132504549743,),
  ('competency_strength_InWeek3',): (0.14281208000704368,),
  ('competency_anticipation_InWeek3',): (0.13262485362853155,),
  ('student_shape_InWeek4',): (0.036183274932754514,),
  ('competency_ant

In [61]:
MC_LIME_results

{}

In [37]:
for key in MC_LIME_results:
    MC_LIME_results[key] = {str(inner_key): list(inner_value) for inner_key, inner_value in MC_LIME_results[key].items()}

# Save MC_LIME_results as a JSON file
with open('MC_LIME_results.json', 'w') as f:
    json.dump(MC_LIME_results, f)

## CEM